In [0]:
from pyspark.sql import SparkSession 

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, LongType, StringType, DateType, IntegerType

In [0]:
import datetime
from datetime import timedelta

In [0]:
spark = SparkSession.builder.appName('FactSalesDimensionality').getOrCreate()

In [0]:
dbutils.fs.ls("/FileStore/")

Out[6]: [FileInfo(path='dbfs:/FileStore/shared_uploads/', name='shared_uploads/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/', name='tables/', size=0, modificationTime=0)]

In [0]:
dbutils.fs.ls("/FileStore/shared_uploads/")


Out[7]: [FileInfo(path='dbfs:/FileStore/shared_uploads/nxd210029@utdallas.edu/', name='nxd210029@utdallas.edu/', size=0, modificationTime=0)]

In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[8]: [FileInfo(path='dbfs:/FileStore/tables/an_utd_python_s3_us_west_1_amazonaws_com_yelp_labelled.txt', name='an_utd_python_s3_us_west_1_amazonaws_com_yelp_labelled.txt', size=61320, modificationTime=1687812855000),
 FileInfo(path='dbfs:/FileStore/tables/crime_data.csv', name='crime_data.csv', size=1440, modificationTime=1687816551000),
 FileInfo(path='dbfs:/FileStore/tables/fact_sales.csv', name='fact_sales.csv', size=2348, modificationTime=1749790542000),
 FileInfo(path='dbfs:/FileStore/tables/students-1.csv', name='students-1.csv', size=94, modificationTime=1749433987000),
 FileInfo(path='dbfs:/FileStore/tables/students-2.csv', name='students-2.csv', size=94, modificationTime=1749437015000),
 FileInfo(path='dbfs:/FileStore/tables/students.csv', name='students.csv', size=94, modificationTime=1749433896000)]

In [0]:
df = spark.read.csv("/FileStore/tables/fact_sales.csv", header=True, inferSchema=True)



In [0]:
df.show()

+------+---------------+------------------+------------+---------------+--------+---------+----------+---------+-------+
|Region|ProductCategory|ProductSubCategory|SalesChannel|CustomerSegment|SalesRep|StoreType| SalesDate|UnitsSold|Revenue|
+------+---------------+------------------+------------+---------------+--------+---------+----------+---------+-------+
| North|    Electronics|            Mobile|      Online|       Consumer|   Alice|    Urban|2023-07-21|       10|   1000|
| South|       Clothing|             Shirt|     Offline|      Corporate|     Bob| Suburban|2024-01-28|       20|   1500|
|  East|      Furniture|             Table|      Retail|    Home Office| Charlie|    Rural|2025-02-22|       15|   1200|
|  West|      Groceries|        Vegetables|   Wholesale| Small Business|   Diana|    Metro|2023-01-24|       30|   3000|
| North|    Electronics|            Mobile|      Online|       Consumer|   Alice|    Urban|2025-01-31|       12|   1100|
| South|       Clothing|        

In [0]:
df_region = df.select("Region").distinct()

In [0]:
df_Region = df_region.withColumn("DIM_RegionId", hash(upper(col("Region"))).cast("bigint"))

In [0]:
df_Region.show()

+------+------------+
|Region|DIM_RegionId|
+------+------------+
| South|   -96241737|
|  East| -1592160435|
|  West|  -402154387|
| North|  1869582694|
+------+------------+



In [0]:
dfBase = spark.createDataFrame([("N/A", -1)], ["Region", "DIM_RegionId"])


In [0]:
df_Region_Final = df_Region.union(dfBase)
df_Region_Final.show()


+------+------------+
|Region|DIM_RegionId|
+------+------------+
| South|   -96241737|
|  East| -1592160435|
|  West|  -402154387|
| North|  1869582694|
|   N/A|          -1|
+------+------------+



In [0]:
df_sales_channel = df.select("SalesChannel").distinct()
df_sales_channel = df_sales_channel.withColumn("DIM_SalesChannelId", hash(upper(col("SalesChannel"))).cast("bigint"))
df_base = spark.createDataFrame([("N/A", -1)], ["SalesChannel", "DIM_SalesChannelId"])
df_sales_channel_final = df_sales_channel.union(df_base)
df_sales_channel_final.show()


+------------+------------------+
|SalesChannel|DIM_SalesChannelId|
+------------+------------------+
|   Wholesale|        2128323039|
|      Online|        2000399776|
|      Retail|       -2034590472|
|     Offline|        1815579165|
|         N/A|                -1|
+------------+------------------+



In [0]:
df_customer = df.select("CustomerSegment").distinct()
df_customer = df_customer.withColumn("DIM_CustomerId", hash(upper(col("CustomerSegment"))).cast("bigint"))
df_base = spark.createDataFrame([("N/A", -1)], ["CustomerSegment", "DIM_CustomerId"])
df_customer_final = df_customer.union(df_base)
df_customer_final.show()


+---------------+--------------+
|CustomerSegment|DIM_CustomerId|
+---------------+--------------+
|       Consumer|   -1594651443|
|    Home Office|    -951279011|
|      Corporate|      -7382621|
| Small Business|     911933479|
|            N/A|            -1|
+---------------+--------------+



In [0]:
df_sales_rep = df.select("SalesRep").distinct()
df_sales_rep = df_sales_rep.withColumn("DIM_SalesRepId", hash(upper(col("SalesRep"))).cast("bigint"))
df_base = spark.createDataFrame([("N/A", -1)], ["SalesRep", "DIM_SalesRepId"])
df_sales_rep_final = df_sales_rep.union(df_base)
df_sales_rep_final.show()


+--------+--------------+
|SalesRep|DIM_SalesRepId|
+--------+--------------+
|   Diana|   -1866580689|
| Charlie|     591137949|
|     Bob|    -392921166|
|   Alice|    1352887388|
|     N/A|            -1|
+--------+--------------+



In [0]:
df_location = df.select("StoreType").distinct()
df_location = df_location.withColumn("DIM_LocationId", hash(upper(col("StoreType"))).cast("bigint"))
df_base = spark.createDataFrame([("N/A", -1)], ["StoreType", "DIM_LocationId"])
df_location_final = df_location.union(df_base)
df_location_final.show()


+---------+--------------+
|StoreType|DIM_LocationId|
+---------+--------------+
|    Urban|   -1427273716|
| Suburban|   -1640598988|
|    Rural|    -817774136|
|    Metro|    1735384348|
|      N/A|            -1|
+---------+--------------+



In [0]:
df_ProductCategory = df.select("ProductCategory").distinct()
df_ProductCategory = df_ProductCategory.withColumn("DIM_ProductCategoryId", hash(upper(col("ProductCategory"))).cast("bigint"))
df_base = spark.createDataFrame([("N/A", -1)], ["ProductCategory", "DIM_ProductCategoryId"])
df_ProductCategory_final = df_ProductCategory.union(df_base)
df_ProductCategory_final.show()

+---------------+---------------------+
|ProductCategory|DIM_ProductCategoryId|
+---------------+---------------------+
|      Groceries|            963552435|
|    Electronics|           1422188909|
|       Clothing|            218386994|
|      Furniture|           -871983438|
|            N/A|                   -1|
+---------------+---------------------+



In [0]:
df_ProductSubCategory = df.select("ProductSubCategory").distinct()
df_ProductSubCategory = df_ProductSubCategory.withColumn("DIM_ProductSubCategoryId", hash(upper(col("ProductSubCategory"))).cast("bigint"))
df_base = spark.createDataFrame([("N/A", -1)], ["ProductSubCategory", "DIM_ProductSubCategoryId"])
df_ProductSubCategory_final = df_ProductSubCategory.union(df_base)
df_ProductSubCategory_final.show()

+------------------+------------------------+
|ProductSubCategory|DIM_ProductSubCategoryId|
+------------------+------------------------+
|        Vegetables|              -167645305|
|             Table|             -1029118937|
|             Shirt|              -686888261|
|            Mobile|             -1314911383|
|               N/A|                      -1|
+------------------+------------------------+



In [0]:
date_range = df.select(min(col("SalesDate")).alias("MinDate"), max(col("SalesDate")).alias("MaxDate")).collect()
min_date = date_range[0]["MinDate"]
max_date = date_range[0]["MaxDate"]
print(min_date, max_date)

2020-06-25 2025-06-04


In [0]:
date_list = []
current_date = min_date
while current_date <= max_date:
    date_list.append((current_date,))
    current_date += timedelta(days=1)


df_dates = spark.createDataFrame(date_list, ["Date"])

In [0]:
df_dates.show()

+----------+
|      Date|
+----------+
|2020-06-25|
|2020-06-26|
|2020-06-27|
|2020-06-28|
|2020-06-29|
|2020-06-30|
|2020-07-01|
|2020-07-02|
|2020-07-03|
|2020-07-04|
|2020-07-05|
|2020-07-06|
|2020-07-07|
|2020-07-08|
|2020-07-09|
|2020-07-10|
|2020-07-11|
|2020-07-12|
|2020-07-13|
|2020-07-14|
+----------+
only showing top 20 rows



In [0]:
df_dates = df_dates.withColumn("DIM_DateID", hash(col("Date")).cast("bigint")) \
                   .withColumn("MonthName", expr("date_format(Date, 'MMMM')")) \
                   .withColumn("Year", expr("year(Date)")) \
                   .withColumn("Semester", expr("CASE WHEN month(Date) IN (1,2,3,4,5,6) THEN 'H1' ELSE 'H2' END")) \
                   .withColumn("Quarter", expr("CASE WHEN month(Date) IN (1,2,3) THEN 'Q1' \
                                                WHEN month(Date) IN (4,5,6) THEN 'Q2' \
                                                WHEN month(Date) IN (7,8,9) THEN 'Q3' \
                                                ELSE 'Q4' END"))


In [0]:
df_dates.schema

Out[26]: StructType([StructField('Date', DateType(), True), StructField('DIM_DateID', LongType(), False), StructField('MonthName', StringType(), True), StructField('Year', IntegerType(), True), StructField('Semester', StringType(), False), StructField('Quarter', StringType(), False)])

In [0]:
schema = StructType([
    StructField("Date", DateType(), True),
    StructField("DIM_DateID", LongType(), False),
    StructField("MonthName", StringType(), True),
    StructField("Year", IntegerType(), True),
    StructField("Semester", StringType(), False),
    StructField("Quarter", StringType(), False)
])

df_base = spark.createDataFrame([(None, -1, "N/A", -1, "N/A", "N/A")], schema=schema)


df_dates_final = df_dates.union(df_base)
display(df_dates_final)


Date,DIM_DateID,MonthName,Year,Semester,Quarter
2020-06-25,803745435,June,2020,H1,Q2
2020-06-26,1082212343,June,2020,H1,Q2
2020-06-27,1584034908,June,2020,H1,Q2
2020-06-28,-1037504348,June,2020,H1,Q2
2020-06-29,-464378206,June,2020,H1,Q2
2020-06-30,4942199,June,2020,H1,Q2
2020-07-01,263318531,July,2020,H2,Q3
2020-07-02,-978791968,July,2020,H2,Q3
2020-07-03,-739675161,July,2020,H2,Q3
2020-07-04,170797161,July,2020,H2,Q3


In [0]:
df_Region_Final.write.format("delta").mode("overwrite").saveAsTable("DIM_Region")
df_sales_channel_final.write.format("delta").mode("overwrite").saveAsTable("DIM_SalesChannel")
df_customer_final.write.format("delta").mode("overwrite").saveAsTable("DIM_Customer")
df_sales_rep_final.write.format("delta").mode("overwrite").saveAsTable("DIM_SalesRep")
df_location_final.write.format("delta").mode("overwrite").saveAsTable("DIM_Location")
df_ProductCategory_final.write.format("delta").mode("overwrite").saveAsTable("DIM_ProductCategory")
df_ProductSubCategory_final.write.format("delta").mode("overwrite").saveAsTable("DIM_ProductSubCategory")
df_dates_final.write.format("delta").mode("overwrite").saveAsTable("DIM_Dates")


In [0]:
df_DIMRegion = spark.table("DIM_Region")
df_DimSalesChannel = spark.table("DIM_SalesChannel")
df_DIMCustomer = spark.table("DIM_Customer")
df_DimSalesRep = spark.table("DIM_SalesRep")
df_DIMLocation = spark.table("DIM_Location")
df_DIMProductCategory = spark.table("DIM_ProductCategory")
df_DIMProductSubCategory = spark.table("DIM_ProductSubCategory")
df_DIMDates = spark.table("DIM_Dates")


In [0]:
display(df_DIMRegion)

Region,DIM_RegionId
South,-96241737
East,-1592160435
West,-402154387
North,1869582694
N/A,-1


##All Tables are loaded

In [0]:
df_fact_clean = (
    df
    .withColumn("DIM_RegionId", when(col("Region").isNull(), -1)
                .otherwise(hash(upper(col("Region")))).cast("bigint"))
    .withColumn("DIM_ProductCategoryId", when(col("ProductCategory").isNull(), -1)
                .otherwise(hash(upper(col("ProductCategory")))).cast("bigint"))
    .withColumn("DIM_ProductSubCategoryId", when(col("ProductSubCategory").isNull(), -1)
                .otherwise(hash(upper(col("ProductSubCategory")))).cast("bigint"))
    .withColumn("DIM_SalesChannelId", when(col("SalesChannel").isNull(), -1)
                .otherwise(hash(upper(col("SalesChannel")))).cast("bigint"))
    .withColumn("DIM_CustomerId", when(col("CustomerSegment").isNull(), -1)
                .otherwise(hash(upper(col("CustomerSegment")))).cast("bigint"))
    .withColumn("DIM_SalesRepId", when(col("SalesRep").isNull(), -1)
                .otherwise(hash(upper(col("SalesRep")))).cast("bigint"))
    .withColumn("DIM_LocationId", when(col("StoreType").isNull(), -1)
                .otherwise(hash(upper(col("StoreType")))).cast("bigint"))
    .withColumn("DIM_DateId", when(col("SalesDate").isNull(), -1)
                .otherwise(hash(upper(col("SalesDate").cast("string")))).cast("bigint"))
    .select("DIM_RegionId", "DIM_ProductCategoryId", "DIM_ProductSubCategoryId",
            "DIM_SalesChannelId", "DIM_CustomerId", "DIM_SalesRepId", "DIM_LocationId", "DIM_DateId",
            "UnitsSold", "Revenue")
)

In [0]:
display(df_fact_clean)

DIM_RegionId,DIM_ProductCategoryId,DIM_ProductSubCategoryId,DIM_SalesChannelId,DIM_CustomerId,DIM_SalesRepId,DIM_LocationId,DIM_DateId,UnitsSold,Revenue
1869582694,1422188909,-1314911383,2000399776,-1594651443,1352887388,-1427273716,715112604,10,1000
-96241737,218386994,-686888261,1815579165,-7382621,-392921166,-1640598988,1128210370,20,1500
-1592160435,-871983438,-1029118937,-2034590472,-951279011,591137949,-817774136,-747075131,15,1200
-402154387,963552435,-167645305,2128323039,911933479,-1866580689,1735384348,-509316217,30,3000
1869582694,1422188909,-1314911383,2000399776,-1594651443,1352887388,-1427273716,1368929520,12,1100
-96241737,218386994,-686888261,1815579165,-7382621,-392921166,-1640598988,657425828,25,2500
-1592160435,-871983438,-1029118937,-2034590472,-951279011,591137949,-817774136,2127210736,22,2200
-402154387,963552435,-167645305,2128323039,911933479,-1866580689,1735384348,-89329102,18,1800
1869582694,1422188909,-1314911383,2000399776,-1594651443,1352887388,-1427273716,-55123064,10,1000
-96241737,218386994,-686888261,1815579165,-7382621,-392921166,-1640598988,1504773733,20,1500


In [0]:
df_fact_clean.select("DIM_ProductCategoryId").distinct().show()

+---------------------+
|DIM_ProductCategoryId|
+---------------------+
|            963552435|
|           1422188909|
|           -871983438|
|            218386994|
+---------------------+



In [0]:
df_DIMProductCategory.select("DIM_ProductCategoryId").distinct().show()

+---------------------+
|DIM_ProductCategoryId|
+---------------------+
|            963552435|
|           1422188909|
|           -871983438|
|            218386994|
|                   -1|
+---------------------+



Above table does not show -1 N/A as The -1 key (for "N/A" or missing values) is deliberately added in each DIM table as a default “unknown” or “not applicable” row.

In [0]:


df_fact_clean1 = (
    df
    .withColumn("DIM_RegionId", when(col("Region").isNull(), lit(-1))
                .otherwise(hash(upper(col("Region")))).cast("bigint"))
    .withColumn("DIM_ProductCategoryId", when(col("ProductCategory").isNull(), lit(-1))
                .otherwise(hash(upper(col("ProductCategory")))).cast("bigint"))
    .withColumn("DIM_ProductSubCategoryId", when(col("ProductSubCategory").isNull(), lit(-1))
                .otherwise(hash(upper(col("ProductSubCategory")))).cast("bigint"))
    .withColumn("DIM_SalesChannelId", when(col("SalesChannel").isNull(), lit(-1))
                .otherwise(hash(upper(col("SalesChannel")))).cast("bigint"))
    .withColumn("DIM_CustomerId", when(col("CustomerSegment").isNull(), lit(-1))
                .otherwise(hash(upper(col("CustomerSegment")))).cast("bigint"))
    .withColumn("DIM_SalesRepId", when(col("SalesRep").isNull(), lit(-1))
                .otherwise(hash(upper(col("SalesRep")))).cast("bigint"))
    .withColumn("DIM_LocationId", when(col("StoreType").isNull(), lit(-1))
                .otherwise(hash(upper(col("StoreType")))).cast("bigint"))
    .withColumn("DIM_DateId", when(col("SalesDate").isNull(), lit(-1))
                .otherwise(hash(col("SalesDate"))).cast("bigint"))
    .select(
        "DIM_RegionId", "DIM_ProductCategoryId", "DIM_ProductSubCategoryId",
        "DIM_SalesChannelId", "DIM_CustomerId", "DIM_SalesRepId",
        "DIM_LocationId", "DIM_DateId", "UnitsSold", "Revenue"
    )
)


In [0]:
display(df_fact_clean1)

DIM_RegionId,DIM_ProductCategoryId,DIM_ProductSubCategoryId,DIM_SalesChannelId,DIM_CustomerId,DIM_SalesRepId,DIM_LocationId,DIM_DateId,UnitsSold,Revenue
1869582694,1422188909,-1314911383,2000399776,-1594651443,1352887388,-1427273716,-448243438,10,1000
-96241737,218386994,-686888261,1815579165,-7382621,-392921166,-1640598988,1428085869,20,1500
-1592160435,-871983438,-1029118937,-2034590472,-951279011,591137949,-817774136,190521420,15,1200
-402154387,963552435,-167645305,2128323039,911933479,-1866580689,1735384348,-2019786856,30,3000
1869582694,1422188909,-1314911383,2000399776,-1594651443,1352887388,-1427273716,29313737,12,1100
-96241737,218386994,-686888261,1815579165,-7382621,-392921166,-1640598988,-1649562627,25,2500
-1592160435,-871983438,-1029118937,-2034590472,-951279011,591137949,-817774136,1689844821,22,2200
-402154387,963552435,-167645305,2128323039,911933479,-1866580689,1735384348,-1760240056,18,1800
1869582694,1422188909,-1314911383,2000399776,-1594651443,1352887388,-1427273716,-1284366460,10,1000
-96241737,218386994,-686888261,1815579165,-7382621,-392921166,-1640598988,-1602584443,20,1500


In [0]:
df_joined = (
    df_fact_clean1
    .join(df_DIMRegion, "DIM_RegionId", "inner")
    .join(df_DIMProductCategory, "DIM_ProductCategoryId", "inner")
    .join(df_DIMProductSubCategory, "DIM_ProductSubCategoryId", "inner")
    .join(df_DimSalesChannel, "DIM_SalesChannelId", "inner")
    .join(df_DIMCustomer, "DIM_CustomerId", "inner")
    .join(df_DimSalesRep, "DIM_SalesRepId", "inner")
    .join(df_DIMLocation, "DIM_LocationId", "inner")
    .join(df_DIMDates, "DIM_DateId", "inner")
)

In [0]:
display(df_joined)

DIM_DateId,DIM_LocationId,DIM_SalesRepId,DIM_CustomerId,DIM_SalesChannelId,DIM_ProductSubCategoryId,DIM_ProductCategoryId,DIM_RegionId,UnitsSold,Revenue,Region,ProductCategory,ProductSubCategory,SalesChannel,CustomerSegment,SalesRep,StoreType,Date,MonthName,Year,Semester,Quarter
805957329,-1640598988,-392921166,-7382621,1815579165,-686888261,218386994,-96241737,25,2500,South,Clothing,Shirt,Offline,Corporate,Bob,Suburban,2021-11-26,November,2021,H2,Q4
-250820072,-1640598988,-392921166,-7382621,1815579165,-686888261,218386994,-96241737,20,1500,South,Clothing,Shirt,Offline,Corporate,Bob,Suburban,2021-11-09,November,2021,H2,Q4
-514062881,-1640598988,-392921166,-7382621,1815579165,-686888261,218386994,-96241737,25,2500,South,Clothing,Shirt,Offline,Corporate,Bob,Suburban,2022-05-29,May,2022,H1,Q2
-935695912,-1640598988,-392921166,-7382621,1815579165,-686888261,218386994,-96241737,20,1500,South,Clothing,Shirt,Offline,Corporate,Bob,Suburban,2022-07-08,July,2022,H2,Q3
1921201188,-1640598988,-392921166,-7382621,1815579165,-686888261,218386994,-96241737,25,2500,South,Clothing,Shirt,Offline,Corporate,Bob,Suburban,2022-12-23,December,2022,H2,Q4
-1602584443,-1640598988,-392921166,-7382621,1815579165,-686888261,218386994,-96241737,20,1500,South,Clothing,Shirt,Offline,Corporate,Bob,Suburban,2024-03-16,March,2024,H1,Q1
-1649562627,-1640598988,-392921166,-7382621,1815579165,-686888261,218386994,-96241737,25,2500,South,Clothing,Shirt,Offline,Corporate,Bob,Suburban,2023-06-27,June,2023,H1,Q2
1428085869,-1640598988,-392921166,-7382621,1815579165,-686888261,218386994,-96241737,20,1500,South,Clothing,Shirt,Offline,Corporate,Bob,Suburban,2024-01-28,January,2024,H1,Q1
1816191791,-817774136,591137949,-951279011,-2034590472,-1029118937,-871983438,-1592160435,15,1200,East,Furniture,Table,Retail,Home Office,Charlie,Rural,2023-03-23,March,2023,H1,Q1
29313737,-817774136,591137949,-951279011,-2034590472,-1029118937,-871983438,-1592160435,22,2200,East,Furniture,Table,Retail,Home Office,Charlie,Rural,2025-01-31,January,2025,H1,Q1
